<a href="https://colab.research.google.com/github/nbeaudoin/PyTorch-marathon/blob/main/Computer_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Computer vision libraries in python

* `tochvision` - base
* `torchvision.datasets` - get datasets adn data loading functions for computer vision
* `torchvision.models` - get pretrained computer vision models that you can leverage for your CV problems
* `tochvision.transforms` - functions for manipulating your vision data (images)
* `toch.utils.data.Dataset` - Create your own datasets
* `torch.utils.data.DataLoader`` - Python iterable over a dataset